In [1]:
from CrySPY.gen_struc.random import rndgen
from xenonpy.datatools import preset

import pandas as pd
from pymatgen import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [2]:
tmp = rndgen.rndgen_spg(
    nstruc=10000, 
    atype=['Li', 'P'], 
    nat=[6, 2],
    spgnum=[194],
    symprec=0.01,
    mindist=[[2.0, 2.0], [2.0, 2.0]], fwpath='/Users/liuchang/source/CrySPY/CrySPY/find_wy/find_wy')

Structure ID        0 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID        1 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID        2 was generated. Space group: 194 --> 191 P6/mmm
Structure ID        3 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID        4 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID        5 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID        6 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID        7 was generated. Space group: 194 --> 191 P6/mmm
Structure ID        8 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID        9 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID       10 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID       11 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID       12 was generated. Space group: 194 --> 194 P6_3/mmc
Structure ID       13 was generated. Space group: 194 --> 194 P6_3/mmc
Structure 

In [3]:
def gen_info(s, id_):
    a, b, c = s.lattice.abc
    alpha, beta, gamma = s.lattice.angles
    volume = s.lattice.volume
    wyckoffs = SpacegroupAnalyzer(s).get_symmetry_dataset()['wyckoffs']
    site_elements = [t.name for t in s.species]

    if wyckoffs == ['f', 'f', 'f', 'f', 'b', 'b', 'c', 'c'] or wyckoffs == ['b', 'b','f', 'f', 'f', 'f',  'c', 'c']:
        matched = True
    else:
        matched = False
        
    return dict(id=id_, a=a, b=b, c=c, alpha=alpha, beta=beta, gamma=gamma, volume=volume, wyckoffs=wyckoffs, site_elements=site_elements, matched=matched)

In [4]:
summary = []

for k, s in tmp.items():
    id_ = f's{k}'
    filename = f'generated_structs/{id_}.cif'
    s.to(fmt='cif', filename=filename, symprec=0.01)
    summary.append(gen_info(s, id_))
        
pd.DataFrame(summary).set_index('id').to_pickle('generated_structs/summary.pd.xz')

# some test

In [5]:
summary = pd.read_pickle('generated_structs/summary.pd.xz')

In [7]:
summary.groupby('matched').count()

,a,b,c,alpha,beta,gamma,volume,wyckoffs,site_elements
matched,,,,,,,,,
False,9642,9642,9642,9642,9642,9642,9642,9642,9642
True,358,358,358,358,358,358,358,358,358


In [12]:
s = Structure.from_file('Li3P_mp-736_computed.cif')
a = SpacegroupAnalyzer(s)

/usr/local/miniconda3/envs/xepy37/lib/python3.7/site-packages/pymatgen/io/cif.py:1101: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [13]:
a.get_symmetry_dataset()['wyckoffs']

['f', 'f', 'f', 'f', 'b', 'b', 'c', 'c']

In [25]:
a = SpacegroupAnalyzer(tmp[0])
a.get_symmetry_dataset()['wyckoffs']

['b', 'b', 'f', 'f', 'f', 'f', 'd', 'd']

In [98]:
s_ = s.sites[0]

In [100]:
s_.to_unit_cell()

PeriodicSite: Li (0.0000, 2.4456, 4.4222) [0.3333, 0.6667, 0.5841]

In [116]:
d = pd.DataFrame(dict(a=[1,np.nan,3,4,5], b=[6,np.nan,8,9,0]))
d

,a,b
0,1.0,6.0
1,NaN,NaN
2,3.0,8.0
3,4.0,9.0
4,5.0,0.0


In [133]:
d_ = d.dropna()
d_

,a,b
0,1.0,6.0
2,3.0,8.0
3,4.0,9.0
4,5.0,0.0


In [134]:
a = d_['a']
b = d_['b']

In [135]:
a

0    1.0
2    3.0
3    4.0
4    5.0
Name: a, dtype: float64

In [136]:
b

0    6.0
2    8.0
3    9.0
4    0.0
Name: b, dtype: float64

In [137]:
c = pd.concat([a, b], ignore_index=True, sort=False, axis=1)
c

,0,1
0,1.0,6.0
2,3.0,8.0
3,4.0,9.0
4,5.0,0.0


In [131]:
pd.DataFrame(c, index=d.index)

,0,1
0,1.0,6.0
1,NaN,NaN
2,3.0,8.0
3,4.0,9.0
4,5.0,0.0
